# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - można zaimplementować w tym celu wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [ ]:
from typing import Tuple, List
import copy
import random
import math
import time

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove, ConnectFourState

Wielkość planszy

In [ ]:
ROW_COUNT = 4
COLUMN_COUNT = 7

In [ ]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour):
        self.game = game
        self.fields = game.state.fields
        self.columns_count = len(self.fields) #Trzeba się niezwykle napracować nad taką głupotą
        self.rows_count = len(self.fields[0])
        # print(self.rows_count)
        # print(self.columns_count)
        self.p_min = game.get_players()[0]
        self.p_max = game.get_players()[1]

    def evaluate_position(self, player: Player)->float:
        
        # game1=copy.deepcopy(self.game)
        pass

    def alpha_beta(self, depth, alpha, beta, is_maximizing_player:bool)-> int:
        """Returns column index and score"""

        if depth == 0:
            return 1
        elif self.get_wining_move()[0] != -1:
            if is_maximizing_player:
                return 1000
            else:
                return -1000
                                
        tab_h = []
        if self.game.get_moves() != None:
            if is_maximizing_player:

                for m in self.game.get_moves():

                    game_c=copy.deepcopy(self.game)
                    game_c.make_move(m)
                    mmsol = MinMaxSolver(game_c)
                    alpha = max(alpha, mmsol.alpha_beta(depth-1,alpha,beta, (not is_maximizing_player)))
                    
                    if alpha >= beta:
                        return beta
                return alpha
            else:
                for m in self.game.get_moves():

                    game_c=copy.deepcopy(self.game)
                    game_c.make_move(m)
                    mmsol = MinMaxSolver(game_c)
                    beta = min(beta, mmsol.alpha_beta(depth-1,alpha,beta, (not is_maximizing_player)))
                    
                    if alpha >= beta:
                        return alpha
                return beta
        
        else:
            return 0
            

    def minimax(self, depth, is_maximizing_player:bool)-> int:
        """Returns best score"""

        if depth == 0:
            return 1
        elif self.get_wining_move()[0] != -1:
            if is_maximizing_player:
                return 1000
            else:
                return -1000
                                
        tab_h = []
        if self.game.get_moves() != None:
            for m in self.game.get_moves():

                game_c=copy.deepcopy(self.game)
                game_c.make_move(m)
                mmsol = MinMaxSolver(game_c)

                # print(game_c)
                # print(depth)
                best_mov_h = mmsol.minimax(depth-1,(not is_maximizing_player))
                # print(best_mov_h)
                tab_h.append(best_mov_h)
            # print(tab_h)
            if is_maximizing_player:
                # max wartosc z drugiej pozycji krotki
                # print("wybieram max")
                tab_h.sort(reverse=True)
            else:
                # print("wybieram min")
                tab_h.sort(reverse=False)
            if len(tab_h) > 0:
                return tab_h[0]
            else:
                return 0
        else:
            return 0
        

    def get_wining_move(self):#returns -1 if wining move do not exist

        moves = self.game.get_moves()
        for m in moves:
            game1=copy.deepcopy(self.game)
            player1 = game1.get_current_player()
            game1.make_move(m)
            # print(game1)
            player2= game1.get_current_player()
            if(game1.get_winner()==player1):
                return [m.column, player1]
            elif(game1.get_winner()==player2):
                return [m.column, player2]
        return [-1, None]

    def get_column_height(self, col: int) -> int:
        i=0
        for i, field in enumerate(self.fields[col]):
            if field is not None:
                i = i + 1
            else:
                return i
                
    def heuristic(self, x, y):
        y = min(y,self.rows_count)
        x = min(x,self.columns_count)
        y_score = math.floor((self.columns_count)/2)-math.floor(abs(y-(self.rows_count-1)/2))
        x_score = math.floor((self.rows_count)/2)-math.floor(abs(x-(self.columns_count-1)/2))
        return x_score+y_score+1
            


In [ ]:
class ConnectFourGame:
    def __init__(self, game: ConnectFour):
        self.game = game

    def Bot_vs_Bot_minmax(self):
        
        depth1 = 5
        depth2 = 4
        if(self.game.get_current_player() == self.game.first_player):
            print("zaczyna a")
            a_turn = True
        else:
            print("zaczyna b")
            a_turn = False

        # A JEST MAXEM, B JEST MINEM!!!

        print("Pozycja początkowa")
        print(self.game)

        while(self.game.get_moves() != None):
            if(a_turn):
                opt_move = self.move(depth1,0,0,(not a_turn),True)
            else:
                opt_move = self.move(depth2,0,0,(not a_turn),True)
            # print(opt_move)
            if(opt_move != None):
                self.game.make_move(ConnectFourMove(opt_move))
            a_turn = not a_turn
            # print(self.game)
            # time.sleep(1)
            
        print(self.game)
        if(self.game.get_winner()==self.game.first_player):
            print("wygrał a")
        elif(self.game.get_winner()!=None):
            print("wygrał b")
        else:
            print("remis")   
    
   
    
    def move(self, depth, alpha:float, beta:float, is_maximizing_player:bool, minmax_algorithm:bool)-> int:
        
        mmsol = MinMaxSolver(self.game)

        if (mmsol.get_wining_move()[0] != -1):
            print("Mam ruch wygrywający")
            return mmsol.get_wining_move()[0]

        if minmax_algorithm:
            best_mov_val = mmsol.minimax(depth, is_maximizing_player)
        else:
            best_mov_val = mmsol.minimax(depth,alpha,beta, is_maximizing_player)

        print("Najwięcej mogę ugrać")
        print(best_mov_val)
        moves = self.game.get_moves()
        
        if best_mov_val == 0:
            # print("remis")
            return 1
        else:
            best_moves = []
            
            for m in moves:
                game_c=copy.deepcopy(self.game)
                game_c.make_move(m)
                mmsol = MinMaxSolver(game_c)
                # print("dla ruchu  "+ str(m.column)+"  mam wynik")
                # print(mmsol.minimax(depth-1, not is_maximizing_player))

                if (mmsol.minimax(depth-1, not is_maximizing_player)) == best_mov_val:
                    # print(m.column)
                    best_moves.append(m.column)
            print("Ruchy ktore pozwalaja mi tyle ugrac")
            print(best_moves)

            # TUTAJ NASTĄPIŁA ZMIANA
            if(best_mov_val == 1):
                max_h = 0
                best_c = 0
                for x in best_moves:
                    # print("col")
                    # print(mmsol.get_column_height(x))
                    temp_h = mmsol.heuristic(x, mmsol.get_column_height(x))
                    # print(temp_h)
                    if  temp_h > max_h:
                        best_c = x
                        max_h = mmsol.heuristic(x,mmsol.get_column_height(x))
                print("czyli wybieram heurystycznie")
            else:
                print("czyli wybieram losowo bo i tak wygralme/przegralem")
                best_c = random.choice(best_moves)

            print(best_c)
            return best_c

    


In [ ]:
#BvB test 2
#BvB test
ROW_COUNT = 3
COLUMN_COUNT = 3
p1 = Player("a")
p2 = Player("b")

game1 = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)

#Test 1 - b obrona w dwoch
game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(0))

game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(0))

game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(2))
print(game1)
print(game1.get_moves())
Conn_Game = ConnectFourGame(game1)
Conn_Game.Bot_vs_Bot_minmax()
# print("opt rozw")
# print(x)

In [ ]:
#BvB test
ROW_COUNT = 5
COLUMN_COUNT = 6
p1 = Player("a")
p2 = Player("b")

game1 = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)


#Test 1 - b obrona w dwoch
game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(2))
# Test 2 - b obrona w jednym
# game1.make_move(ConnectFourMove(1))
# game1.make_move(ConnectFourMove(2))
# game1.make_move(ConnectFourMove(3))
# game1.make_move(ConnectFourMove(4))
# game1.make_move(ConnectFourMove(2))
# game1.make_move(ConnectFourMove(3))
# game1.make_move(ConnectFourMove(3))
# game1.make_move(ConnectFourMove(4))
# game1.make_move(ConnectFourMove(4))


print(game1)
Conn_Game = ConnectFourGame(game1)
Conn_Game.Bot_vs_Bot_minmax()
# print("opt rozw")
# print(x)

Pierwszy test pokazuje, że b (ale tylko posiadający depth >= 4 !!!) jest w stanie obronić się przed wygraną a w dwóch ruchach. Przy b mniej "inteligentnym" a wygrywa w swoim drugim ruchu.  Drugi test pokazuje, że b widzi wygrywający ruch a i blokuje to pole.
Dla testów przy różnej głębokości przeszukiwania a i b, "zawodnik" o głębszym przeszukiwaniu czasem wygrywał, jednak ze względu na wybór heurystki faworyzującej środkowe pola, gdy najczęściej kończą się remisem.

In [ ]:
#Minmax i Alpha-Beta test
ROW_COUNT = 5
COLUMN_COUNT = 5
p1 = Player("a")
p2 = Player("b")

game1 = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)

#Test 1
# game1.make_move(ConnectFourMove(1))
# game1.make_move(ConnectFourMove(1))
# game1.make_move(ConnectFourMove(2))
#Test 2
# game1.make_move(ConnectFourMove(1))
# game1.make_move(ConnectFourMove(2))
# game1.make_move(ConnectFourMove(1))
# game1.make_move(ConnectFourMove(3))
# game1.make_move(ConnectFourMove(1))
# game1.make_move(ConnectFourMove(2))
#Test3
# game1.make_move(ConnectFourMove(1))
# game1.make_move(ConnectFourMove(2))
# game1.make_move(ConnectFourMove(3))
# game1.make_move(ConnectFourMove(4))
# game1.make_move(ConnectFourMove(2))
# game1.make_move(ConnectFourMove(3))
# game1.make_move(ConnectFourMove(3))
# game1.make_move(ConnectFourMove(4))
# game1.make_move(ConnectFourMove(4))
# game1.make_move(ConnectFourMove(0))
# game1.make_move(ConnectFourMove(1))
# Test4
game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(2))
game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(3))
game1.make_move(ConnectFourMove(0))

a_move = True
a_move = not a_move
print(game1)
minmax = MinMaxSolver(game1)
x = minmax.minimax(3,a_move)
print("największy możliwy wynik")
print("minimax")
print(x)

ab = minmax.alpha_beta(3,-10000,10000,a_move)
print("alpha beta")
print(ab)

TEST HEURYSTYKI

In [ ]:
ROW_COUNT = 8
COLUMN_COUNT = 10
p1 = Player("a")
p2 = Player("b")
t = [ [0]*8 for i in range(10)]
game_0 =ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
minmax_0 = MinMaxSolver(game_0)
for i in range(COLUMN_COUNT):
    for j in range(ROW_COUNT):
        t[i][j] = minmax_0.heuristic(i,j)
print(t)
print(t[9][0])


In [ ]:

game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(4))


print(game.get_winner() == p1)
print(game.get_winner() == p2)
game.make_move(ConnectFourMove(2))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))
print(game.get_winner() == p1)
print(game.get_winner() == p2)


moves = game.get_moves()
fields = game.state.fields
m1 = moves[0]
print(m1.column)
print(len(moves))
print("col height check")

print(game)